In [72]:
import pandas as pd

In [140]:
def add_vvp(dataframe, col_date ='date', file_source = 'VVP.csv'):
    """
    передаем датафрейм и название колонки с датой, к которой идет привязка ВВП
    формат колонки с датой должен быть datetime64 и с часами
    """
    # читаем файл с динамикой ВВП
    vvp = pd.read_csv(file_source)
    # преобразуем дату файла-источника в формат datetime64 и дропнем один столбик
    vvp['date'] = pd.to_datetime(vvp['date'], format ='%Y-%m-%d %H:%M:%S')
    vvp.drop('for_month',axis=1,inplace=True) 
    # соединяем основной фрейм и ВВП по дате объявления показтеля ВВП
    result = pd.merge(dataframe,vvp,how ='left', on='date')
    # считаем, что пока нет новостей, то берем старые данные
    result['VVP_perc'].fillna(method = 'ffill',inplace=True)
    # для начала датасета берем показатель ВВП из 2018 года вручную
    result['VVP_perc'].fillna(value = vvp['VVP_perc'].iloc[-1]
                              ,inplace=True  )
    # возращаем новый датасет 
    return result

In [145]:
# пример работы
data = pd.read_csv('train_dataset.csv')
data['date'] = pd.to_datetime(data['date'], format = '%Y-%m-%d' )
# соединяем дату и часы в один datatime 
def date_plus_hours_to_index(data):
    data = data.copy()
    data['date'] = data['date'] + pd.to_timedelta(data['time'] , 'H')
    data = data.set_index('date')
    return data
# применяем функцию добавления ВВП
example = add_vvp(data,'date')

In [146]:
example.head()

date  time   target  temp  temp_pred weather_pred weather_fact  \
0 2019-01-01     0  481.510   2.9        2.0  пасм, ветер        ветер   
1 2019-01-01     1  462.872   2.9        2.0  пасм, ветер        ветер   
2 2019-01-01     2  449.718   2.9        2.0  пасм, ветер        ветер   
3 2019-01-01     3  430.908   4.3        2.0  пасм, ветер  ветер, пасм   
4 2019-01-01     4  415.163   4.3        2.0  пасм, ветер  ветер, пасм   

   VVP_perc  
0       1.8  
1       1.8  
2       1.8  
3       1.8  
4       1.8